In [1]:
import sqlite3
import pandas as pd
import numpy as np
import time
from utils import *
from ReactNode import ReactNode
from ChemNode import ChemNode
from MCTS import MCTS

In [2]:
conn = sqlite3.connect('buyable.db')
buyable = conn.cursor()
retrobiocat = pd.read_pickle("data/retrobiocat_database.pkl")
analyzer = Retrosim()

In [3]:
smile = "C[C@@H](N)[C@H](O)c1ccccc1"
root = ChemNode(smile, 0, None, True, buyable, retrobiocat, analyzer, None)

In [4]:
mcts = MCTS(root, 1000)

In [5]:
start = time.time()
mcts.MCTS()
end = time.time()

Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing selected
Nothing select

In [6]:
for react in root.reactions:
    print(react.reaction_name)
    for precursor in react.precursors:
        print(f'Chem {precursor.smiles}, Score: {precursor.score}, visits: {precursor.visits}, solution: {precursor.solution}')
    print(" ")

Ester hydrolysis (OH)
Chem CCC(=O)O[C@H](c1ccccc1)[C@@H](C)N, Score: 0.0, visits: 1, solution: False
 
Ketone amination
Chem CC(=O)[C@H](O)c1ccccc1, Score: 0.0, visits: 1, solution: False
 
Amide hydrolysis (NH)
Chem CC(=O)N[C@H](C)[C@H](O)c1ccccc1, Score: 0.0, visits: 1, solution: False
 
Ketone reduction
Chem C[C@@H](N)C(=O)c1ccccc1, Score: 1.0, visits: 1, solution: True
 
Phosphate hydrolysis
Chem C[C@@H](N)[C@H](OP(=O)(O)O)c1ccccc1, Score: 0.0, visits: 1, solution: False
 
Imine reduction
Chem CC(=N)[C@H](O)c1ccccc1, Score: 0.0, visits: 1, solution: False
 
Imine reduction
Chem CC(=N)[C@H](O)c1ccccc1, Score: 0.0, visits: 1, solution: False
 


In [7]:
root.solution

True

In [8]:
def prune_tree(node: ChemNode):
    """
    Prune the tree by removing all non-solution pathways.

    :param node: The root ChemNode to prune.
    :return: None (modifies the tree in-place).
    """
    # Iterate over a copy of the reactions to safely modify the original list
    for reaction in node.reactions[:]:
        # Check if all reagents in the reaction are solutions
        if all(reagent.solution for reagent in reaction.precursors):
            # Recursively prune each reagent
            for reagent in reaction.precursors:
                prune_tree(reagent)
        else:
            # Remove non-solution reagents from the reaction
            for reagent in reaction.precursors[:]:
                if not reagent.solution:
                    reaction.precursors.remove(reagent)
            
            # If the reaction has no reagents left, remove the reaction
            if not reaction.precursors:
                node.reactions.remove(reaction)

    # If the node itself is not a solution and has no reactions, prune it at the parent level
    if not node.solution and not node.reactions:
        return False

# Plan: Go down tree, copying along a given path to add to a single path list

def generate_subtrees(node: ChemNode):
    # Base case: if the node is a leaf
    if node.solution and not node.reactions: # Leaf node
        node.solution = False
        return node.copy()
    
    copy = node.copy()
    # If the node is not a leaf, generate subtrees for all reactions
       
    reaction = node.reactions[0]
    react = reaction.copy()
    for reagent in reaction.precursors:
        if reagent.solution:
            react.add_reagent(generate_subtrees(reagent))

    if all(not reagent.solution for reagent in node.reactions[0].precursors):
        node.reactions.pop(0)

    if not node.reactions:
        node.solution = False

    

    copy.reactions.append(react)
    return copy

def generate_paths(root: ChemNode):
    paths = []
    while root.solution:
        paths.append(generate_subtrees(root))
    return paths

def path_explorer(root: ChemNode):
    """
    Prints the path given a subtree
    """

    stack = []
    stack.append(root)
    count = 0
    while stack:
        print(f"BRANCH {count}:\n-----------------")
        count += 1
        root = stack.pop()
        print(f'Chem: {root.smiles}')
        while root.reactions:
            print(f'Reaction: {root.reactions[0].reaction_name}')
            if len(root.reactions[0].precursors) > 1:
                print(f"SPLIT: Chem1: {root.reactions[0].precursors[0].smiles}, Chem2: {root.reactions[0].precursors[1].smiles}\n")
                stack.append(root.reactions[0].precursors[1])
            else:
                print(f'Chem: {root.reactions[0].precursors[0].smiles}')
            root = root.reactions[0].precursors[0]
        print("BUYABLE\n")
        

In [9]:
# Prune the tree starting from the root
prune_tree(root)
paths = generate_paths(root)
print(len(paths))

1


In [10]:
path_explorer(paths[0])

BRANCH 0:
-----------------
Chem: C[C@@H](N)[C@H](O)c1ccccc1
Reaction: Ketone reduction
Chem: C[C@@H](N)C(=O)c1ccccc1
BUYABLE

